In [12]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.regression import gmm

# Read the CSV file and parse dates in the first column
file_path = 'data.csv'
data = pd.read_csv(file_path, parse_dates=[0])

# Print the first five rows of the dataframe
print(data.head())


  Unnamed: 0  year  Cnsmr  Manuf  HiTec   Hlth  Other  SPCSPI    IR  \
0          1  1927  26.40  31.78  37.88  56.54  37.34   13.40  4.26   
1          2  1928  38.18  51.25  59.14  32.58  30.57   17.53  4.64   
2          3  1929 -43.01 -27.61  -6.18 -17.51 -22.02   24.86  6.01   
3          4  1930 -35.04 -39.94 -39.83 -17.87 -39.68   21.71  4.15   
4          5  1931 -35.16 -48.58 -48.64 -37.21 -53.03   15.98  2.43   

   pc_consumption  
0     5760.196756  
1     5818.535581  
2     6071.470984  
3     5686.178628  
4     5469.072499  


In [13]:
# Calculate the consumption growth and lagged values
data['c_growth'] = data['pc_consumption'] / data['pc_consumption'].shift(1)
data['c_growth_lag1'] = data['c_growth'].shift(1)
data['r_lag1'] = data['IR'].shift(1)
data['r_lag2'] = data['IR'].shift(2)
data['r_forw1'] = data['IR'].shift(-1)
data['c_lag1'] = data['pc_consumption'].shift(1)
data['c_forw1'] = data['pc_consumption'].shift(-1)
data['const'] = 1

# Remove rows with missing values
data_clean = data.dropna()


# Define endogenous, exogenous, and instrumental variables
endog_df = data_clean[['r_forw1', 'c_forw1', 'pc_consumption']]
exog_df = endog_df
instrument_df = data_clean[['r_lag1', 'r_lag2', 'c_growth', 'c_growth_lag1', 'const']]

endog, exog, instrument = map(np.asarray, [endog_df, exog_df, instrument_df])
print(data.head())

  Unnamed: 0  year  Cnsmr  Manuf  HiTec   Hlth  Other  SPCSPI    IR  \
0          1  1927  26.40  31.78  37.88  56.54  37.34   13.40  4.26   
1          2  1928  38.18  51.25  59.14  32.58  30.57   17.53  4.64   
2          3  1929 -43.01 -27.61  -6.18 -17.51 -22.02   24.86  6.01   
3          4  1930 -35.04 -39.94 -39.83 -17.87 -39.68   21.71  4.15   
4          5  1931 -35.16 -48.58 -48.64 -37.21 -53.03   15.98  2.43   

   pc_consumption  c_growth  c_growth_lag1  r_lag1  r_lag2  r_forw1  \
0     5760.196756       NaN            NaN     NaN     NaN     4.64   
1     5818.535581  1.010128            NaN    4.26     NaN     6.01   
2     6071.470984  1.043471       1.010128    4.64    4.26     4.15   
3     5686.178628  0.936541       1.043471    6.01    4.64     2.43   
4     5469.072499  0.961819       0.936541    4.15    6.01     3.36   

        c_lag1      c_forw1  const  
0          NaN  5818.535581      1  
1  5760.196756  6071.470984      1  
2  5818.535581  5686.178628      1 

In [14]:
def moment_consumption1(params, exog):
    beta, gamma = params
    r_forw1, c_forw1, c = exog.T  # unwrap iterable (ndarray)

    # moment condition without instrument
    err = 1 - beta * (1 + r_forw1) * np.power(c_forw1 / c, -gamma)
    return -err

In [30]:
endog1 = np.zeros(exog.shape[0])
mod1 = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=4)
w0inv = np.dot(instrument.T, instrument) / len(endog1)
res1 = mod1.fit([1, -1], maxiter=10, inv_weights=w0inv)

Optimization terminated successfully.
         Current function value: 0.137095
         Iterations: 17
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.217435
         Iterations: 8
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: 0.141138
         Iterations: 4
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.136910
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.138006
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.138761
         Iterations: 4
         Function evaluations: 10
       

In [32]:
I = np.eye(instrument.shape[1])
res2 = mod1.fit([1, -1], maxiter=10, inv_weights=I)

Optimization terminated successfully.
         Current function value: 0.083831
         Iterations: 32
         Function evaluations: 41
         Gradient evaluations: 41
Optimization terminated successfully.
         Current function value: 0.184497
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.081200
         Iterations: 9
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.142861
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.121309
         Iterations: 6
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.129647
         Iterations: 5
         Function evaluations: 11
       

In [33]:
print(res1.summary(yname='Euler Eq', xname=['discount', 'CRRA']))


                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        11.18
Model:                 NonlinearIVGMM   Prob (Hansen J):               0.00373
Method:                           GMM                                         
Date:                Sat, 29 Apr 2023                                         
Time:                        17:14:04                                         
No. Observations:                  80                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
discount       0.2055      0.023      8.800      0.000       0.160       0.251
CRRA          19.9791      6.567      3.043      0.002       7.109      32.849


In [34]:
print(res2.summary(yname='Euler Eq', xname=['discount', 'CRRA']))

                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        11.06
Model:                 NonlinearIVGMM   Prob (Hansen J):               0.00397
Method:                           GMM                                         
Date:                Sat, 29 Apr 2023                                         
Time:                        17:14:25                                         
No. Observations:                  80                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
discount       0.2056      0.023      8.795      0.000       0.160       0.251
CRRA          20.1265      6.582      3.058      0.002       7.226      33.027


In [70]:

# We don't need Nelder-Mead in this case, we can use bfgs default directly
#res1_ = mod1.fit([1,-1], maxiter=0, inv_weights=w0inv, opt_method='nm')

res1_hac4_2s = mod1.fit([1, -1], maxiter=10, inv_weights=w0inv, weights_method="cov")#'hac', wargs={'maxlag':6})
print('\n\n')
print(res1_hac4_2s.summary(yname='Euler Eq', xname=['discount', 'CRRA']))

Optimization terminated successfully.
         Current function value: 0.137095
         Iterations: 17
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.217435
         Iterations: 8
         Function evaluations: 14
         Gradient evaluations: 14
Optimization terminated successfully.
         Current function value: 0.141138
         Iterations: 4
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.136910
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.138006
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.138761
         Iterations: 4
         Function evaluations: 10
       

In [62]:
# Define endogenous, exogenous, and instrumental variables
endog_df = data_clean[['r_forw1', 'c_forw1', 'pc_consumption']]
exog_df = endog_df
instrument_df = data_clean[[ 'c_growth']]

endog1 = np.zeros(exog.shape[0])
mod1 = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=6)
I = np.eye(instrument.shape[1])
res1 = mod1.fit([1, -1], maxiter=10, inv_weights=I)
print(res1.summary(yname='Euler Eq', xname=['discount', 'CRRA']))

Optimization terminated successfully.
         Current function value: 0.083831
         Iterations: 32
         Function evaluations: 41
         Gradient evaluations: 41
Optimization terminated successfully.
         Current function value: 0.184497
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.081200
         Iterations: 9
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.142861
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.121309
         Iterations: 6
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.129647
         Iterations: 5
         Function evaluations: 11
       

                            NonlinearIVGMM Results                            
Dep. Variable:               Euler Eq   Hansen J:                        11.06
Model:                 NonlinearIVGMM   Prob (Hansen J):                0.0259
Method:                           GMM                                         
Date:                Sun, 30 Apr 2023                                         
Time:                        17:49:57                                         
No. Observations:                  80                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
discount       0.2056      0.023      8.795      0.000       0.160       0.251
CRRA          20.1265      6.582      3.058      0.002       7.226      33.027


In [50]:
data.head()

,Unnamed: 0,year,Cnsmr,Manuf,HiTec,Hlth,Other,SPCSPI,IR,pc_consumption,c_growth,c_growth_lag1,r_lag1,r_lag2,r_forw1,c_lag1,c_forw1,const
0,1,1927,26.40,31.78,37.88,56.54,37.34,13.40,4.26,5760.196756,NaN,NaN,NaN,NaN,4.64,NaN,5818.535581,1
1,2,1928,38.18,51.25,59.14,32.58,30.57,17.53,4.64,5818.535581,1.010128,NaN,4.26,NaN,6.01,5760.196756,6071.470984,1
2,3,1929,-43.01,-27.61,-6.18,-17.51,-22.02,24.86,6.01,6071.470984,1.043471,1.010128,4.64,4.26,4.15,5818.535581,5686.178628,1
3,4,1930,-35.04,-39.94,-39.83,-17.87,-39.68,21.71,4.15,5686.178628,0.936541,1.043471,6.01,4.64,2.43,6071.470984,5469.072499,1
4,5,1931,-35.16,-48.58,-48.64,-37.21,-53.03,15.98,2.43,5469.072499,0.961819,0.936541,4.15,6.01,3.36,5686.178628,4951.900583,1


In [95]:
import numpy as np
import pandas as pd
from statsmodels.sandbox.regression import gmm

def prepare_instruments(data, num_lags):
    for lag in range(1, num_lags + 1):
        data[f'c_growth_lag{lag}'] = data['c_growth'].shift(lag)
    return data

def estimate_gmm(data, num_lags):
    dta = prepare_instruments(data, num_lags)
    
    dta_clean = dta.dropna()
    
    endog_df = dta_clean[['r_forw1', 'c_forw1', 'pc_consumption']]
    exog_df = endog_df
    instrument_columns = ['r_lag1', 'r_lag2', 'const'] + [f'c_growth_lag{i}' for i in range(1, num_lags + 1)]
    instrument_df = dta_clean[instrument_columns]

    endog, exog, instrument = map(np.asarray, [endog_df, exog_df, instrument_df])

    endog1 = np.zeros(exog.shape[0])

    mod1 = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=num_lags + 3)
    w0inv = np.dot(instrument.T, instrument) / len(endog1)
    res1 = mod1.fit([1, -1], maxiter=2, inv_weights=w0inv)

    return res1.params

# Read the CSV file and parse dates in the first column
file_path = 'data.csv'
dta = pd.read_csv(file_path, parse_dates=[0])

# Calculate additional columns needed for the analysis
dta['c_growth'] = dta['pc_consumption'] / dta['pc_consumption'].shift(1)
dta['r_lag1'] = dta['IR'].shift(1)
dta['r_lag2'] = dta['IR'].shift(2)
dta['r_forw1'] = dta['IR'].shift(-1)
dta['c_lag1'] = dta['pc_consumption'].shift(1)
dta['c_forw1'] = dta['pc_consumption'].shift(-1)
dta['const'] = 1

# Define the moment function
def moment_consumption1(params, exog):
    beta, gamma = params
    r_forw1, c_forw1, c = exog.T  # unwrap iterable (ndarray)

    # Moment condition without instrument
    err = 1 - beta * (1 + r_forw1) * np.power(c_forw1 / c, -gamma)
    return -err

# List of lags to be tested
lags_to_test = [1, 2, 4, 6]

# Estimate GMM for each number of lags and store the results
results = [estimate_gmm(dta, num_lags) for num_lags in lags_to_test]

# Create a DataFrame to display the results
result_df = pd.DataFrame(results, columns=['discount', 'CRRA'], index=lags_to_test)
result_df.index.name = 'lags'

# Print the results
print(result_df)

KeyError: 'pc_consumption'

IndexError: tuple index out of range

In [108]:
dta_clean

,Unnamed: 0,year,Cnsmr,Manuf,HiTec,Hlth,Other,SPCSPI,IR,pc_consumption,c_growth,c_growth_lag1,r_lag1,r_lag2,r_forw1,c_lag1,c_forw1,const
2,3,1929,-43.01,-27.61,-6.18,-17.51,-22.02,24.86,6.01,6071.470984,1.043471,1.010128,4.64,4.26,4.150,5818.535581,5686.178628,1
3,4,1930,-35.04,-39.94,-39.83,-17.87,-39.68,21.71,4.15,5686.178628,0.936541,1.043471,6.01,4.64,2.430,6071.470984,5469.072499,1
4,5,1931,-35.16,-48.58,-48.64,-37.21,-53.03,15.98,2.43,5469.072499,0.961819,0.936541,4.15,6.01,3.360,5686.178628,4951.900583,1
5,6,1932,8.99,17.17,-7.37,-14.08,10.33,8.30,3.36,4951.900583,0.905437,0.961819,2.43,4.15,1.460,5469.072499,4815.926239,1
6,7,1933,151.29,139.30,137.15,17.40,115.39,7.09,1.46,4815.926239,0.972541,0.905437,3.36,2.43,1.010,4951.900583,5123.656959,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,78,2004,23.23,34.17,11.63,15.95,23.72,1132.52,1.49,29087.295635,1.025331,1.018679,1.18,1.85,3.410,28368.686990,29790.295775,1
78,79,2005,2.72,17.62,-3.53,-3.51,6.01,1181.41,3.41,29790.295775,1.024169,1.025331,1.49,1.18,5.320,29087.295635,30364.433438,1
79,80,2006,18.47,21.36,12.98,11.33,17.34,1278.73,5.32,30364.433438,1.019273,1.024169,3.41,1.49,5.340,29790.295775,30867.609872,1
80,81,2007,-12.67,8.27,-0.55,-4.84,-16.79,1424.16,5.34,30867.609872,1.016571,1.019273,5.32,3.41,3.420,30364.433438,30509.081237,1


In [102]:
# Read the CSV file and parse dates in the first column
file_path = 'data.csv'
dta = pd.read_csv(file_path, parse_dates=[0])
# Print the first five rows of the dataframe
print(dta.head())

  Unnamed: 0  year  Cnsmr  Manuf  HiTec   Hlth  Other  SPCSPI    IR   CPI  \
0          1  1927  26.40  31.78  37.88  56.54  37.34   13.40  4.26  17.5   
1          2  1928  38.18  51.25  59.14  32.58  30.57   17.53  4.64  17.3   
2          3  1929 -43.01 -27.61  -6.18 -17.51 -22.02   24.86  6.01  17.1   
3          4  1930 -35.04 -39.94 -39.83 -17.87 -39.68   21.71  4.15  17.1   
4          5  1931 -35.16 -48.58 -48.64 -37.21 -53.03   15.98  2.43  15.9   

   Real_1yr  real_pc_consumption  real_SP_return  
0  1.054653          5760.196756        0.381460  
1  1.058639          5818.535581        0.483782  
2  1.060100          6071.470984       -0.087691  
3  1.120104          5686.178628       -0.159834  
4  1.138907          5469.072499       -0.365431  


In [104]:
industries = ['Cnsmr', 'Manuf', "HiTec", 'Hlth', 'Other']
dta['cpi_change'] = dta['CPI'] / dta['CPI'].shift(1)-1
for industry in industries:
    dta['real'+industry] = dta[industry]/100- dta['cpi_change']
print(dta.head())

  Unnamed: 0  year  Cnsmr  Manuf  HiTec   Hlth  Other  SPCSPI    IR   CPI  \
0          1  1927  26.40  31.78  37.88  56.54  37.34   13.40  4.26  17.5   
1          2  1928  38.18  51.25  59.14  32.58  30.57   17.53  4.64  17.3   
2          3  1929 -43.01 -27.61  -6.18 -17.51 -22.02   24.86  6.01  17.1   
3          4  1930 -35.04 -39.94 -39.83 -17.87 -39.68   21.71  4.15  17.1   
4          5  1931 -35.16 -48.58 -48.64 -37.21 -53.03   15.98  2.43  15.9   

   Real_1yr  real_pc_consumption  real_SP_return  cpi_change  realCnsmr  \
0  1.054653          5760.196756        0.381460         NaN        NaN   
1  1.058639          5818.535581        0.483782   -0.011429   0.393229   
2  1.060100          6071.470984       -0.087691   -0.011561  -0.418539   
3  1.120104          5686.178628       -0.159834    0.000000  -0.350400   
4  1.138907          5469.072499       -0.365431   -0.070175  -0.281425   

   realManuf  realHiTec  realHlth  realOther  
0        NaN        NaN       NaN      

In [113]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
from statsmodels.sandbox.regression import gmm

def prepare_instruments(data, num_lags):
    for lag in range(1, num_lags + 1):
        data[f'c_growth_lag{lag}'] = data['c_growth'].shift(lag)
    return data

# Define the moment function
def moment_consumption1(params, exog):
    beta, gamma = params
    r_forw1, c_forw1, c = exog.T  # unwrap iterable (ndarray)

    # Moment condition without instrument
    err = 1 - beta * (1 + r_forw1) * np.power(c_forw1 / c, -gamma)
    return -err

def estimate_gmm(data, num_lags):
    dta = prepare_instruments(data, num_lags)
    
    dta_clean = dta.dropna()
    
    endog_df = dta_clean[['r_forw1', 'c_forw1', 'real_pc_consumption']]
    exog_df = endog_df
    instrument_columns = ['const'] + [f'c_growth_lag{i}' for i in range(1, num_lags + 1)]
    instrument_df = dta_clean[instrument_columns]

    endog, exog, instrument = map(np.asarray, [endog_df, exog_df, instrument_df])

    endog1 = np.zeros(exog.shape[0])

    mod1 = gmm.NonlinearIVGMM(endog1, exog, instrument, moment_consumption1, k_moms=num_lags + 1)
    w0inv = np.dot(instrument.T, instrument) / len(endog1)
    I = np.eye(instrument.shape[1])  #identity matrix
    res1 = mod1.fit([1, -1], maxiter=10, inv_weights=I)

    
    j_stat, p_value, df = res1.jtest()# Degrees of freedom for the J-test

    
    return res1.params, res1.bse, (j_stat, p_value, df)

# Read the CSV file and parse dates in the first column
file_path = 'data.csv'
dta = pd.read_csv(file_path, parse_dates=[0])

industries = ['Cnsmr', 'Manuf', "HiTec", 'Hlth', 'Other']
dta['cpi_change'] = dta['CPI'] / dta['CPI'].shift(1)-1
for industry in industries:
    dta['real'+industry] = dta[industry]/100- dta['cpi_change']


# George Note: for now please change real_SP_return into "real"+industries if you want to run industry portfolio
    
# Calculate additional columns needed for the analysis
dta['c_growth'] = dta['real_pc_consumption'] / dta['real_pc_consumption'].shift(1)-1
dta['r_lag1'] = dta['real_SP_return'].shift(1)
dta['r_lag2'] = dta['real_SP_return'].shift(2)
dta['r_forw1'] = dta['real_SP_return'].shift(-1)
dta['c_lag1'] = dta['real_pc_consumption'].shift(1)
dta['c_forw1'] = dta['real_pc_consumption'].shift(-1)
dta['const'] = 1



# List of lags to be tested
lags_to_test = [1, 2, 4, 6]

# Estimate GMM for each number of lags and store the results
results = [estimate_gmm(dta, num_lags) for num_lags in lags_to_test]

# Create DataFrames to display the results
params_df = pd.DataFrame([res[0] for res in results], columns=['discount', 'CRRA'], index=lags_to_test)
bse_df = pd.DataFrame([res[1] for res in results], columns=['SE_discount', 'SE_CRRA'], index=lags_to_test)

jtest_df = pd.DataFrame([{'J-stat': res[2][0], 'p-value': res[2][1], 'df': res[2][2]} for res in results], index=lags_to_test)

# Combine the DataFrames to display the results
result_df = pd.concat([params_df, bse_df, jtest_df], axis=1)
result_df.index.name = 'lags'

# Print the results
print(result_df)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.010950
         Iterations: 9
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.003808
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.003830
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.003830
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000002
         Iterations: 1
         Function evaluations: 3
         Gradient